# INM707 Coursework - Advanced Task
# Space Invaders with Deep Q-Network (DQN)
### By: Elisabeta Monica Furdui: 190045971 and Jasveen Kaur: 190020638 

### Importing the required modules 
For this implementation, we have used OpenAI gym and Pytorch library. 

In [ ]:
###############
#Code referenced and edited from: https://notebooks.azure.com/paulamarinhozago1/project/
###############

import numpy as np
import torch
import torch.nn as nn
import torch.autograd as autograd
import time
import torch.nn.functional as F

import gym
from gym import spaces, wrappers
import cv2
import random, math
from collections import deque
from IPython import display
import matplotlib.pyplot as plt
import pickle
import logging
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### OpenAI Wrappers library
To preprocess the input image, stack the frames and reset the environment.

In [ ]:
#Wrappers Library taken and edited from https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.py
class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal, then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert somtimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic, and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype='uint8')
        self._skip = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255, shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]

class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0], shp[1], shp[2] * k), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames

    def __array__(self, dtype=None):
        out = np.concatenate(self._frames, axis=2)
        if dtype is not None:
            out = out.astype(dtype)
        return out

class ImageToPyTorch(gym.ObservationWrapper):
    """
    Change image shape to CWH
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]), dtype=env.observation_space.dtype)

    def observation(self, observation):
        return np.swapaxes(observation, 2, 0)

def make_atari(env_id):
    env = gym.make(env_id)
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False, pytorch_img=False):
    """Configure environment for DeepMind-style Atari.
    Rescale the image to 84 x 84, single gray scale channel, and stack 4 consecutive frames as one observation.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    #if clip_rewards:
     #   env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    if pytorch_img:
        env = ImageToPyTorch(env)
    return env

### Initializing the game environment to space invaders.
Using Monitor to get the video output and Deepmind's wrappers function for preprocessing. 
Applying preprocessing steps from the above wrapper classes and displaying observation and action space.

In [ ]:
env = make_atari('SpaceInvadersNoFrameskip-v4')
env = wrappers.Monitor(env, "./tmp/SpaceInvadersNoFrameskip-v4", force = True)
print("Original Observation space:", env.observation_space)
env = wrap_deepmind(env, frame_stack=True, pytorch_img=True)
print("Observation space after preprocessing:", env.observation_space)
print("\nNumber of actions available to the agent:", env.action_space.n)
action_space = [a for a in range(env.action_space.n)]
print("Actions available to the agent:", env.unwrapped.get_action_meanings())

### Replay Buffer for experience replay: Used to store experiences for training the agent

It stores the experiences of the agent at each time-step in the form of a tuple, and then randomly selects the given experiences. Randomization is important to remove correlations in the consecutive samples of observation space and smoothly reflect changes in data sampling, making the training data diverse. It makes the learning process more efficient.  

In [ ]:
class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append([state, action, reward, next_state, done])
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

### Epsilon Greedy Strategy: Used to maintain a balance between exploration and exploitation
Exploration: agent explores the environment by taking a random action from the given state.
Exploitation: agent exploits the environment by taking the best action from the previously learned ones.

In [ ]:
def get_epsilon(initial_epsilon, final_epsilon, epsilon_decay, frame_idx):
    return final_epsilon + (initial_epsilon - final_epsilon) * math.exp(-1.0 * frame_idx / epsilon_decay)

### Defining the Loss functions used
Huber loss: Smooth_l1_loss in Pytorch with beta, it is less sensitive to outliers in data as compared to MSE.
MSE: Mean sqaured error loss
MAE: Mean Absolute Error loss

In [ ]:
def Huber_loss(input, target, beta=1, size_average=True):
    abs_error = torch.abs(input - target)
    cond = x < beta
    loss = torch.where(cond, 0.5 * abs_error ** 2 / beta, abs_error - 0.5 * beta)
    if size_average:
        return loss.mean()
    return loss.sum()

def mse_loss(input, target):
    squared_error = (input - target) ** 2
    sum_squared_error = torch.sum(squared_error)
    loss = sum_squared_error.mean()
    return loss

def mae_loss(input, target):
    abs_error = torch.abs(input - target)
    sum_abs_error = torch.sum(abs_error)
    loss = sum_abs_error / target.size
    return loss

### Deep Neural Network architecture
The convolutional layer is used to directly extract features from the images of the screen

In [ ]:
class Deep_Q_Network(nn.Module):
    def __init__(self):
        super(Deep_Q_Network, self).__init__()
        
        self.conv = nn.Sequential(nn.Conv2d(4, 32, kernel_size=8, stride=4), nn.ReLU(),
                                  nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(),
                                  nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU())
        self.fc = nn.Sequential(nn.Linear(7 * 7 * 64, 512), nn.ReLU(), nn.Linear(512, n_actions))

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
        x = self.fc(x)
        return x #Q values for each action

#### Policy and Target network 
Target Network: It prevents the action-values from oscillating and the algorithm from diverging. Hence making the training and learning process more stable.

In [ ]:
n_actions = len(action_space)
lr = 0.0001
alpha = 0.95

policy_network = Deep_Q_Network().to(device)
target_network = Deep_Q_Network().to(device)
target_network.load_state_dict(policy_network.state_dict())

#optimizer = torch.optim.RMSprop(policy_network.parameters(), lr=lr, alpha=alpha)
optimizer = torch.optim.Adam(policy_network.parameters(), lr=lr)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(policy_network)
print("Number of learnable parameters: %d" % count_parameters(policy_network))

#### Logger to save the logs for rewards

In [ ]:
logger = logging.getLogger('dqn_spaceinvaders')
logger.setLevel(logging.INFO)
logger_handler = logging.FileHandler('./dqn_spaceinvaders_log.log')
logger.addHandler(logger_handler)

#### Defining the parameters to train the model

In [ ]:
max_episodes = 20000
batch_size = 32
target_update = 5000
gamma = 0.99
buffer_size = 500000
buffer_initialize = 50000
skip_frame = 3
initial_epsilon = 1.0
final_epsilon = 0.1
epsilon_decay = 250000

#### Replay Buffer Initialization

In [ ]:
replay_buffer = ReplayBuffer(buffer_size)

while len(replay_buffer) < buffer_initialize:
    
    observation = env.reset()
    done = False
    
    while not done:
        with torch.no_grad():
            obs_n = torch.from_numpy(observation).float().to(device)
            obs_n = obs_n.view(1, obs_n.shape[0], obs_n.shape[1], obs_n.shape[2])
            action = random.sample(range(len(action_space)), 1)[0]
        
        next_observation, reward, done, info = env.step(action_space[action])
            
        replay_buffer.push(observation, action, reward, next_observation, done)
        observation = next_observation
        
print('Replay buffer initialized')

#### Training the Agent by updating the states and actions and calculating the rewards and loss values

In [ ]:
episode_score = []
mean_episode_score = []
mean_episode_loss = []
loss_value = []
num_frames = 0
episode = 0
score = 0
start_time = time.time()

while episode < max_episodes:
    
    observation = env.reset()
    done = False
    
    while not done:
        
        with torch.no_grad():

            obs_n = torch.from_numpy(observation).float().to(device) / 255
            obs_n = obs_n.view(1, obs_n.shape[0], obs_n.shape[1], obs_n.shape[2])
            epsilon = get_epsilon(initial_epsilon, final_epsilon, epsilon_decay, num_frames)
            
            if random.random() > epsilon:
                q_value = policy_network(obs_n)
                action = q_value.argmax(1).data.cpu().numpy().astype(int)[0]
            else:
                action = random.sample(range(len(action_space)), 1)[0]
        
        next_observation, reward, done, info = env.step(action_space[action])
        
        num_frames += 1
        score += reward
        
        #adding experiences to the replay buffer
        replay_buffer.push(observation, action, reward, next_observation, done)
        observation = next_observation
        
        #updating the policy network
        if len(replay_buffer) > batch_size and num_frames % skip_frame == 0:
            
            observations, actions, rewards, next_observations, dones = replay_buffer.sample(batch_size)          

            observations = torch.from_numpy(np.array(observations) / 255).float().to(device)
            next_observations = torch.from_numpy(np.array(next_observations) / 255).float().to(device)
            actions = torch.from_numpy(np.array(actions).astype(int)).float().to(device)
            actions = actions.view(actions.shape[0], 1)
            rewards = torch.from_numpy(np.array(rewards)).float().to(device)
            rewards = rewards.view(rewards.shape[0], 1)
            dones = torch.from_numpy(np.array(dones).astype(int)).float().to(device)
            dones = dones.view(dones.shape[0], 1)
            
            q_values = policy_network(observations)
            next_q_values = target_model(next_observations)

            q_value = q_values.gather(1, actions.long())
            next_q_value = next_q_values.max(1)[0].unsqueeze(1)
            expected_q_value = rewards + gamma * next_q_value * (1 - dones)
            
            #calculating the loss
            
            loss = huber_loss(q_value, expected_q_value)
            #loss = F.SmoothL1Loss(q_value, expected_q_value)
            #loss = F.mae_loss(q_value, expected_q_value)
            #loss = F.mse_loss(q_value, expected_q_value)
            
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
            
            loss_value.append(loss)
            
        # Updating the target network
        if num_frames % target_update == 0:
            target_network.load_state_dict(policy_network.state_dict())
    
    
    episode += 1
    episode_score.append(score)
   
    if info['ale.lives'] == 0:
        score = 0
    
    if episode % 100 == 0:
        mean_score = np.mean(episode_score)
        mean_loss = np.mean(loss_value, dtype = float)
        
        mean_episode_score.append(mean_score)
        mean_episode_loss.append(mean_loss)
        
        episode_score = []
        loss_value = []
        
        data = "Time: {} |  Average Episode Reward (last 100 episodes): {} |  Average Loss per batch (last 100 episodes): {} |  Episode: {}".format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - start_time)), mean_score, '%.4f'%mean_loss, episode)
        print(data)
        
        logger.info(' | Episode: ' + str(episode) + ' | Average Episode Reward (last 100 episodes): ' + str(int(mean_score)) + '| Loss:' + str(float('%.4f'%mean_loss)) +'| Time: ' + time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - start_time)) + '| Frame:' + str(num_frames))  
        
        #saving the loss and rewards as pickle file
        pickle.dump(mean_episode_loss, open('./mean_episode_loss.pickle','wb'))
        pickle.dump(mean_episode_score, open('./mean_episode_scores.pickle', 'wb'))
        
    if episode % 1000 == 0:
        torch.save(policy_network.state_dict(), './trained_model.pt')